In [99]:
# ====================================================#
# Import Package & Setting Environment
# ====================================================#

import dgl as d
import torch as t
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from utils import QoSGraphDataset, FeatureLookup
import logging
import numpy as np
from tqdm import *
import dgl.function as fn
log = logging.getLogger('log')
import pandas as pd
t.set_default_tensor_type(t.FloatTensor)

def Metrics(realVec, estiVec):
    realVec = np.array(realVec)
    estiVec = np.array(estiVec)

    absError = np.abs(estiVec - realVec)
    mae = np.mean(absError)
    nmae = mae / (np.sum(realVec) / absError.shape[0])
    rmse = np.linalg.norm(absError) / np.sqrt(absError.shape[0])
    relativeError = absError / realVec
    mre = np.percentile(relativeError, 50)
    npre = np.percentile(relativeError, 90)

    return np.array([mae, nmae, rmse, mre, npre])

In [100]:
# ====================================================#
# Experiment Setting
# ====================================================#
params = {
    'density':[0.10],
    'dimension':[32],
    'order':[1,2,3],
    'epochs':15,
    'batchsize':256,
    'testsize':400000,
    'type':'rt'
}

In [101]:
class A:
    def __init__(self, ):
        self.dataset = 'rt'
        self.path = './datasets/data/WSDREAM/'
        self.density = 0.10
        self.processed = 0
        self.part_type = 1
        self.slices = 1
        self.epochs = 30
        self.retrain = 1
        self.batch_size = 128
# args = A()

In [102]:
from datasets.dataloader import get_dataloaders
from datasets.dataset import load_data, ShardedTensorDataset
from utility.utils import *

# 数据模型读取
# df = np.array(load_data(args))
# log('\t原始数据集读取完毕')
#
# dataset = ShardedTensorDataset(df, True, args)
# log('\t分切数据执行完毕')
#
# datasets = dataset.get_tensor(0)
#
#
# train_loader, valid_loader, test_loader = get_dataloaders(datasets, False, args)


In [103]:
# ====================================================#
# Create Graph
# ====================================================#
def create_graph():
    print('Registering Nodes')
    userg = d.DGLGraph()
    servg = d.DGLGraph()
    user_lookup = FeatureLookup()
    serv_lookup = FeatureLookup()
    ufile = pd.read_csv('./datasets/data/WSDREAM/原始数据/userlist_table.csv')
    ufile = pd.DataFrame(ufile)
    ulines = ufile.to_numpy()
    ulines = ulines

    sfile = pd.read_csv('./datasets/data/WSDREAM/原始数据/wslist_table.csv')
    sfile = pd.DataFrame(sfile)
    slines = sfile.to_numpy()
    slines = slines

    for i in range(339):
        user_lookup.register('User',i)
    for j in range(5825):
        serv_lookup.register('Serv', j)
    for ure in ulines[:,2]:
        user_lookup.register('URE', ure)
    for uas in ulines[:, 4]:
        user_lookup.register('UAS', uas)

    for sre in slines[:, 4]:
        serv_lookup.register('SRE', sre)
    for spr in slines[:, 2]:
        serv_lookup.register('SPR', spr)
    for sas in slines[:, 6]:
        serv_lookup.register('SAS', sas)

    userg.add_nodes(len(user_lookup))
    servg.add_nodes(len(serv_lookup))

    for line in ulines:
        uid = line[0]
        ure = user_lookup.query_id(line[2])
        if not userg.has_edge_between(uid,ure):
            userg.add_edge(uid, ure)

        uas = user_lookup.query_id(line[4])
        if not userg.has_edge_between(uid,uas):
            userg.add_edge(uid, uas)

    for line in slines:
        sid = line[0]
        sre = serv_lookup.query_id(line[4])
        if not servg.has_edge_between(sid,sre):
            servg.add_edge(sid, sre)

        sas = serv_lookup.query_id(line[6])
        if not servg.has_edge_between(sid,sas):
            servg.add_edge(sid, sas)

        spr = serv_lookup.query_id(line[2])
        if not servg.has_edge_between(sid,spr):
            servg.add_edge(sid, spr)

    userg = d.add_self_loop(userg)
    userg = d.to_bidirected(userg)
    servg = d.add_self_loop(servg)
    servg = d.to_bidirected(servg)
    return user_lookup,serv_lookup,userg,servg

In [104]:
# ====================================================#
# Create Model
# ====================================================#
from dgl.nn import SAGEConv, GATConv
class NeuralCF(t.nn.Module):

    def __init__(self, input_size):
        super(NeuralCF, self).__init__()
        self.layers = t.nn.Sequential(
            t.nn.Linear(input_size, 128),
            t.nn.LayerNorm(128),
            t.nn.ReLU(),
            t.nn.Linear(128, 128),
            t.nn.LayerNorm(128),
            t.nn.ReLU(),
            t.nn.Linear(128, 1)
        )

    def forward(self, input):
        return self.layers(input)


class GraphSAGEConv(t.nn.Module):

    def __init__(self, graph, dim, order=3):
        super(GraphSAGEConv, self).__init__()
        self.order = order
        self.graph = graph
        self.embedding = t.nn.Parameter(t.Tensor(self.graph.number_of_nodes(), dim))
        t.nn.init.kaiming_normal_(self.embedding)
        self.graph.ndata['L0'] = self.embedding
        self.layers = t.nn.ModuleList([SAGEConv(dim, dim, aggregator_type='gcn') for _ in range(order)])
        # self.layers = t.nn.ModuleList([GATConv(dim, dim, num_heads=1) for _ in range(order)])
        self.norms = t.nn.ModuleList([t.nn.LayerNorm(dim) for _ in range(order)])
        self.acts = t.nn.ModuleList([t.nn.RELU() for _ in range(order)])

    def forward(self,uid):
        g = self.graph
        feats = g.ndata['L0']
        for i, (layer, norm, act) in enumerate(zip(self.layers, self.norms, self.acts)):
            feats = layer(g,feats).squeeze()
            feats = norm(feats)
            feats = act(feats)
            g.ndata[f'L{i + 1}'] = feats
        embeds = g.ndata['L0'][uid]
        # for i in range(self.order):
        #     embeds = t.cat((embeds, g.ndata[f'L{i + 1}'][uid]), dim=-1)
        # return feats[uid]
        return embeds


class NeuGraphMF(t.nn.Module):

    def __init__(self, usergraph, servgraph, dim, order=3):
        super(NeuGraphMF, self).__init__()
        self.usergraph = usergraph
        self.servgraph = servgraph
        self.dim = dim
        self.order = order
        self.UserEmbedding = GraphSAGEConv(usergraph, dim, order)
        self.ServEmbedding = GraphSAGEConv(servgraph, dim, order)
        self.ncf = NeuralCF(2 * (order + 1) * dim)

    def forward(self, uid, sid):
        user_embeds = self.UserEmbedding(uid)
        serv_embeds = self.ServEmbedding(sid)
        #return t.sum(user_embeds * serv_embeds,-1).sigmoid()
        return self.ncf(t.cat((user_embeds, serv_embeds), dim=-1)).sigmoid()


In [105]:
def EvaluationOnce(dimension,dataset,order,userg,servg,trainLoader,testLoader, params, args):
    cuda = t.cuda.is_available()
    model = NeuGraphMF(userg, servg, dim=dimension, order=order)
    # print(model)
    #model = t.nn.DataParallel(model)
    if cuda:
        model = model.cuda()

    loss = t.nn.L1Loss().cuda()
    lr = 1e-3
    optimizer = t.optim.AdamW(model.parameters(),lr=lr)

    for epoch in range(args.epochs):
        total_loss = 0
        if epoch % 5 == 0:
            lr /= 2
            optimizer = t.optim.AdamW(model.parameters(),lr=lr,weight_decay=1e-3)

        model.train()
        try:
            with tqdm(trainLoader) as dataiter:
                for batch in dataiter:
                    optimizer.zero_grad()
                    uid,sid,label = batch
                    if cuda:
                        uid = uid.cuda(non_blocking=True)
                        sid = sid.cuda(non_blocking=True)
                        label = label.cuda(non_blocking=True)
                    val = model(uid,sid)
                    #print(val,label)
                    val = val.reshape(label.shape)
                    trainloss = loss(val,label)
                    trainloss.backward()
                    optimizer.step()
                    total_loss += trainloss
        except KeyboardInterrupt:
            dataiter.close()
            raise
        arr0 = np.zeros((5,))
        model.eval()
        try:
            if epoch >= 0:
                with tqdm(testLoader) as dataiter:
                    for batch in dataiter:
                        uid,sid,label = batch
                        if cuda:
                            uid = uid.cuda(non_blocking=True)
                            sid = sid.cuda(non_blocking=True)
                        val = model(uid,sid)
                        if params['type'] == 'rt':
                            val,label = val * dataset.max_value ,label * dataset.max_value
                        else:
                            val, label = val * dataset.max_value, label * dataset.max_value
                        val = val.reshape(label.shape).detach().cpu().numpy()
                        label = label.detach().numpy()
                        arr0 += Metrics(val,label)
        except KeyboardInterrupt:
            dataiter.close()
            raise
        arr0 /= len(testLoader)
        import time
        print(f'Epochs {epoch}: Loss={total_loss / len(trainLoader)}')
        print('Epochs %d: MAE:%.4f NMAE:%.4f RMSE:%.4f MRE:%.4f NPRE:%.4f' %(epoch,*arr0))

def Evaluation(params):
    for density in [0.10]:
        args = A()
        user_lookup, serv_lookup, userg, servg = create_graph()
        df = np.array(load_data(args))
        dataset = ShardedTensorDataset(df, True, args)
        datasets = dataset.get_tensor(0)
        train_loader, valid_loader, test_loader = get_dataloaders(datasets, False, args)
        for dimension in [32]:
            for order in [0]:
                print(f'Evaluating : GraphNeuMF-Dim:{dimension}-Order:{order} in MD={density}')
                EvaluationOnce(dimension=dimension,dataset=dataset, order=order,userg=userg,servg=servg,
                               trainLoader=train_loader,testLoader=test_loader,params=params, args = args)

In [106]:
Evaluation(params=params)

Registering Nodes
2023-01-15 14:23:13  	SISA 切割完毕
Evaluating : GraphNeuMF-Dim:32-Order:0 in MD=0.1


100%|██████████| 1555/1555 [00:09<00:00, 163.32it/s]


Epochs 0: Loss=0.036781489849090576
Epochs 0: MAE:0.6252 NMAE:1.1296 RMSE:1.6022 MRE:0.4820 NPRE:1.8549


100%|██████████| 1555/1555 [00:09<00:00, 164.22it/s]


Epochs 1: Loss=0.026427004486322403
Epochs 1: MAE:0.4788 NMAE:0.8014 RMSE:1.3236 MRE:0.4441 NPRE:1.5008


...

Epochs 27: Loss=0.01564422994852066
Epochs 27: MAE:0.4138 NMAE:0.6024 RMSE:1.1708 MRE:0.3607 NPRE:1.1147


100%|██████████| 1555/1555 [00:09<00:00, 164.02it/s]


Epochs 28: Loss=0.015625467523932457
Epochs 28: MAE:0.4140 NMAE:0.6034 RMSE:1.1712 MRE:0.3619 NPRE:1.1191


100%|██████████| 1555/1555 [00:09<00:00, 161.49it/s]

Epochs 29: Loss=0.015603703446686268
Epochs 29: MAE:0.4138 NMAE:0.6049 RMSE:1.1713 MRE:0.3618 NPRE:1.1210
